<a href="https://colab.research.google.com/github/Ricardo19-art/Investigaci-n-de-Operaciones-/blob/main/Inventario_de_varios_art%C3%ADculos_con_limitaci%C3%B3n_de_almac%C3%A9n.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Inventario de varios artículos con limitación de almacén (EOQ multiartículo)
**Referencia:** Taha, Investigación de Operaciones, Sección 11.2.3 (7a ed.)

## Objetivo
Implementar el modelo de **cantidad económica de pedido (EOQ)** para **varios artículos** con una **restricción de capacidad de almacén**, siguiendo el **método de 3 pasos**:
1. Resolver EOQ sin restricción para cada artículo.
2. Verificar si la solución cumple la restricción de almacén.
3. Si no cumple, resolver el problema con restricción usando optimización (SciPy).

## Modelo
Para artículos $i=1,\dots,n$:

- Demanda anual: $D_i$
- Costo fijo por pedido: $K_i$
- Costo anual de mantener inventario por unidad: $h_i$
- Espacio por unidad: $a_i$
- Capacidad total: $A$

### Función objetivo (costo total anual)

$$
TC(\mathbf{Q}) = \sum_{i=1}^{n}
\left(
\frac{K_i D_i}{Q_i}
+
\frac{h_i Q_i}{2}
\right)
$$

### Restricción de almacén

$$
\sum_{i=1}^{n} a_i Q_i \le A,
\qquad Q_i > 0
$$

In [1]:
import numpy as np
import sympy as sp
from scipy.optimize import minimize

### Método en 3 pasos

**Paso 1:** Si NO hubiera restricción, cada artículo tiene EOQ:

$$
Q_i^* = \sqrt{\frac{2K_i D_i}{h_i}}
$$

**Paso 2:** Se calcula el espacio total:

$$
\sum_{i=1}^n a_i Q_i
$$

Si

$$
\sum_{i=1}^n a_i Q_i \le A
$$

entonces la solución es óptima (la restricción no es activa).

**Paso 3:** Si

$$
\sum_{i=1}^n a_i Q_i > A
$$

la restricción es activa y se resuelve:

$$
\min_{\mathbf{Q}>0} TC(\mathbf{Q})
\quad \text{s.a.} \quad
\sum_{i=1}^n a_i Q_i \le A
$$

usando **SciPy** para encontrar la solución óptima.

In [2]:
def eoq_vector(D, K, h):
    """
    EOQ sin restricción para varios artículos:
    Q_i = sqrt(2 K_i D_i / h_i)
    """
    return np.sqrt((2.0 * K * D) / h)

def costo_total(Q, D, K, h):
    """
    Costo total anual:
    sum( K_i D_i / Q_i + h_i Q_i / 2 )
    Nota: Q debe ser positivo.
    """
    Q = np.asarray(Q, dtype=float)
    return np.sum((K * D) / Q + (h * Q) / 2.0)

def espacio_usado(Q, a):
    """Espacio total usado: sum(a_i Q_i)."""
    return float(np.dot(a, Q))

## Datos de prueba (ejemplo)
Estos datos sirven como caso de prueba para verificar:
- cálculo de EOQ sin restricción,
- verificación de capacidad,
- solución óptima con restricción (si aplica).


In [3]:
D = np.array([1000, 1500], dtype=float)   # demanda anual
K = np.array([100, 120], dtype=float)     # costo por pedido
h = np.array([2, 3], dtype=float)         # costo anual de mantener
a = np.array([1, 1.5], dtype=float)       # espacio por unidad
A = 2000.0                                # capacidad total

In [4]:
Q_eoq = eoq_vector(D, K, h)
Q_eoq

array([316.22776602, 346.41016151])

In [5]:
esp_eoq = espacio_usado(Q_eoq, a)
TC_eoq = costo_total(Q_eoq, D, K, h)

esp_eoq, TC_eoq

(835.8430082875011, np.float64(1671.6860165750022))

## Interpretación (Paso 2)
- Si el espacio usado por EOQ sin restricción **cumple** $ \sum a_iQ_i \le A $, entonces:
  - la restricción no afecta
  - la solución EOQ es aceptable y óptima.
- Si **no cumple**, entonces debemos optimizar con restricción (Paso 3).

In [6]:
# Restricción: A - sum(a_i Q_i) >= 0
constraints = [{
    "type": "ineq",
    "fun": lambda Q: A - np.dot(a, Q)
}]

# Cotas: Q_i >= 1 (evita división entre 0 y valores no físicos)
bounds = [(1.0, None)] * len(D)

# Punto inicial: EOQ (aunque no cumpla, es buena referencia)
Q0 = Q_eoq.copy()

res = minimize(
    fun=costo_total,
    x0=Q0,
    args=(D, K, h),
    method="SLSQP",
    bounds=bounds,
    constraints=constraints
)

res.success, res.message

(True, 'Optimization terminated successfully')

In [7]:
Q_opt = res.x
TC_opt = costo_total(Q_opt, D, K, h)
esp_opt = espacio_usado(Q_opt, a)

print("=== SOLUCIÓN EOQ (sin restricción) ===")
print("Q_eoq:", Q_eoq)
print("Espacio usado:", esp_eoq, " / Capacidad:", A)
print("Costo total:", TC_eoq)

print("\n=== SOLUCIÓN ÓPTIMA (con restricción) ===")
print("Q_opt:", Q_opt)
print("Espacio usado:", esp_opt, " / Capacidad:", A)
print("Costo total:", TC_opt)

=== SOLUCIÓN EOQ (sin restricción) ===
Q_eoq: [316.22776602 346.41016151]
Espacio usado: 835.8430082875011  / Capacidad: 2000.0
Costo total: 1671.6860165750022

=== SOLUCIÓN ÓPTIMA (con restricción) ===
Q_opt: [316.22776602 346.41016151]
Espacio usado: 835.8430082875011  / Capacidad: 2000.0
Costo total: 1671.6860165750022


## Comentarios finales
- El EOQ por artículo es óptimo cuando no hay acoplamiento entre decisiones.  
- La restricción de almacén **acopla** los $Q_i$, porque todos compiten por la misma capacidad $A$.
- Por ello, si el EOQ “libre” viola la capacidad, se requiere resolver un problema de optimización restringida.
- En este notebook:
  - Se implementa el cálculo EOQ con la fórmula cerrada.
  - Se verifica la restricción de capacidad.
  - Se resuelve el problema restringido con **SciPy (SLSQP)**.